In [1]:
#Snowpark lib
#from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN
import pickle

np.random.seed(0)

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
table_name = 'MEMBER_FUNDS_ENRICHED_DETAILS'

In [5]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [6]:
df = sf_df.to_pandas()

In [7]:
df.head()

,MEMBER_ID,MEMBER_NAME,MEMBER_EMPLOYMENT,MEMBER_GENDER,MEMBER_CITY_TOWN,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,FUND_ID,CHURN_REASON,LATITUDE,...,MEMBER_AGE,INVESTMENT_AGE_GROUP,RETIREMENT_AGE,RETIREMENT_AGE_GROUP,TOTAL_FUNDS_INVESTED,NO_OF_CHURN,MEMBER_DOB_DT,ALLOCATION_DT,CHURN_DT,RETIREMENT_DT
0,MID108406,Maile Treston,Charles Darwin University,Male,Lakes Entrance,Victoria,Y,FID000044,NOT APPLICABLE,-37.8667,...,52,Senior,13,MORE_THAN_10_YEAR,3,2.0,1972-04-29,2011-02-25,9999-12-31,2037-04-29
1,MID108415,Annabelle Cryer,RMIT University,Female,Mooroopna,Victoria,N,FID000016,NOT APPLICABLE,-36.3833,...,26,Young,39,MORE_THAN_10_YEAR,3,2.0,1998-10-05,2010-01-04,9999-12-31,2063-10-05
2,MID108429,Leonora Acuff,University of South Australia,Male,Jindabyne,New South Wales,N,FID000100,NOT APPLICABLE,-36.4000,...,51,Senior,14,MORE_THAN_10_YEAR,3,2.0,1973-12-20,2010-09-04,9999-12-31,2038-12-20
3,MID108436,Franklyn Gesick,The University of Newcastle,Male,Wingham,New South Wales,N,FID000012,NOT APPLICABLE,-31.8667,...,41,Middle-aged,24,MORE_THAN_10_YEAR,3,2.0,1983-03-14,2013-04-27,9999-12-31,2048-03-14
4,MID108438,Amber Gillaspie,University of South Australia,Female,Nathalia,Victoria,Y,FID000035,NOT APPLICABLE,-36.0583,...,40,Middle-aged,25,MORE_THAN_10_YEAR,3,2.0,1984-01-06,2012-05-17,9999-12-31,2049-01-06


In [8]:
with open('churn_model.pkl','rb') as f:  ## use rb while reading the fie
    churn_model = pickle.load(f)

In [9]:
df = df.drop(["MEMBER_DOB_DT", 'ALLOCATION_DT', "CHURN_DT", "RETIREMENT_DT"], axis =1 )

In [10]:
frame = df[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

X = frame.drop("CHURN_FLAG", axis=1)
y = frame["CHURN_FLAG"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
@scoring_func
def score(model, request):
    payload_dict = eval(request.json["payload"])
    data = pd.DataFrame.from_dict(payload_dict)
    frame = data[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED']].copy()
    result = model.predict(frame)
    prediction = pd.DataFrame({"CHURN_PREDICTION":result})
    prediction["CHURN_PREDICTION"] = prediction["CHURN_PREDICTION"].apply(lambda x: "Y" if x == 1 else "N")
    probability = model.predict_proba(frame)
    prediction["Probability"] = [round(k[1],4) for k in probability]
    prediction = str(prediction.to_dict())
    return prediction

In [ ]:
df = session.table("MEMBER_FUNDS_ENRICHED_DETAILS").to_pandas()
df = df.drop(["MEMBER_DOB_DT", 'ALLOCATION_DT', "CHURN_DT", "RETIREMENT_DT"], axis =1 )

In [ ]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=churn_model, 
    session=my_session,
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    y_pred=y_pred,
    source="Notebook",
    dataset_name="AUTO_INSURANCE_CLAIMS_DATA",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="Claims_Fraud_DTree_Classifier",
    description="Insurance claims model trained for Fraud Classification",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["scikit-learn==1.3.2"]
)

In [20]:
## registering the model in Fosfor.
model_reg = register_model(churn_model,
               score, 
               name="CHURN_BINARY_MODEL", 
               description="PREDICTING_FUND_CHURN",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install --q fosforml==1.0.1b1",
               y_true=y_test,
               y_pred=y_pred,
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


# CHURN REASON MODEL

In [12]:
with open('reason_model.pkl','rb') as f:  ## use rb while reading the fie
    reason_model = pickle.load(f)

In [13]:
churned = df[df["CHURN_FLAG"] == "Y"]

In [15]:
r2id = {'NO REASON IDENTIFIED': 0,
 'HIGH ACCOUNT FEE': 1,
 'POOR FUND PERFORMANCE': 2,
 'HIGH TRANSACTION FEE': 3,
 'FUND REPUTATION DECLINING': 4,
 'LIFE EVENT': 5}
id2r = {v:k for k, v in r2id.items()}
id2r

{0: 'NO REASON IDENTIFIED',
 1: 'HIGH ACCOUNT FEE',
 2: 'POOR FUND PERFORMANCE',
 3: 'HIGH TRANSACTION FEE',
 4: 'FUND REPUTATION DECLINING',
 5: 'LIFE EVENT'}

In [26]:
@scoring_func
def score(model, request):
    payload_dict = eval(request.json["payload"])
    data = pd.DataFrame.from_dict(payload_dict)
    frame = data[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED']].copy()
    id2r = {0: 'NO REASON IDENTIFIED',
             1: 'HIGH ACCOUNT FEE',
             2: 'POOR FUND PERFORMANCE',
             3: 'HIGH TRANSACTION FEE',
             4: 'FUND REPUTATION DECLINING',
             5: 'LIFE EVENT'}
    result = model.predict(frame)
    prediction = pd.DataFrame({"REASON_PREDICTION":result})
    prediction["REASON_PREDICTION"] = prediction["REASON_PREDICTION"].apply(lambda x: id2r[x])
    prediction = str(prediction.to_dict())
    return prediction

In [27]:
payload = churned.head(10).to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req
yo = score(reason_model, y)
yo

"{'REASON_PREDICTION': {0: 'NO REASON IDENTIFIED', 1: 'NO REASON IDENTIFIED', 2: 'HIGH ACCOUNT FEE', 3: 'POOR FUND PERFORMANCE', 4: 'HIGH TRANSACTION FEE', 5: 'HIGH TRANSACTION FEE', 6: 'FUND REPUTATION DECLINING', 7: 'FUND REPUTATION DECLINING', 8: 'HIGH TRANSACTION FEE', 9: 'FUND REPUTATION DECLINING'}}"

In [31]:

X = churned.drop("CHURN_REASON", axis=1)
y = churned["CHURN_REASON"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [37]:
payload = X_test.to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req
pred_dic = score(reason_model, y)

In [38]:
temp_ = eval(pred_dic)
print("string to dict")
pred = pd.DataFrame.from_dict(temp_)

string to dict


In [39]:
y_pred = pred["REASON_PREDICTION"]
# y_prob = pred["Probability"]

In [43]:
y_prob = reason_model.predict_proba(X_test)

In [40]:
y_test

189394             HIGH ACCOUNT FEE
117948         HIGH TRANSACTION FEE
122069         NO REASON IDENTIFIED
182181             HIGH ACCOUNT FEE
197148    FUND REPUTATION DECLINING
                    ...            
254364         HIGH TRANSACTION FEE
159041    FUND REPUTATION DECLINING
213647         NO REASON IDENTIFIED
310248         NO REASON IDENTIFIED
174585         NO REASON IDENTIFIED
Name: CHURN_REASON, Length: 18491, dtype: object

In [41]:
y_pred

0                 HIGH ACCOUNT FEE
1             HIGH TRANSACTION FEE
2             NO REASON IDENTIFIED
3                 HIGH ACCOUNT FEE
4        FUND REPUTATION DECLINING
                   ...            
18486         HIGH TRANSACTION FEE
18487    FUND REPUTATION DECLINING
18488         NO REASON IDENTIFIED
18489         NO REASON IDENTIFIED
18490         NO REASON IDENTIFIED
Name: REASON_PREDICTION, Length: 18491, dtype: object

In [47]:
## registering the model in Fosfor.
model_reg = register_model(reason_model,
               score, 
               name="CHURN_REASON_MODEL", 
               description="PREDICTING_FUND_CHURN_REASON",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install --q fosforml==1.0.1b1",
               y_true=y_test,
               y_pred=y_pred,
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
